In [90]:
import typing
import functools
from dataclasses import dataclass
from multiprocessing import Pool

import re

In [86]:
@dataclass
class SpringReport:
    state : str
    damaged : typing.List

    @classmethod
    def fromrow(cls,row,repeat=1):
        state,log = row.split()
        return cls("?".join(state for k in range(repeat)),[int(chunk) for chunk in log.split(',')]*repeat)
    
    def totalnumcombinations(self): #Reality check
        return 2**self.state.count("?")
    
    def dumbcompatiblestates(self):
        cnt=0
        for s in generatestates(self.state):
            if [len(chunk) for chunk in s.split(".") if chunk] == self.damaged: cnt+=1
        return cnt
    
    def smartcompatible(self):
        return generatecompatiblestates(self.state,self.damaged[:])
    
    def smartercompatible(self):
        return smartercompatiblestates(self.state,tuple(self.damaged[:]))

    
def generatestates(quantumstate):
    if quantumstate.find("?")>=0:
        yield from generatestates(quantumstate.replace("?",".",1))
        yield from generatestates(quantumstate.replace("?","#",1))
    else:
        yield quantumstate

def generatecompatiblestates(quantumstate,damaged):
    res = 0

    if quantumstate.count("?")==0:
        if [len(chunk) for chunk in quantumstate.split(".") if chunk] == damaged: return 1
        else:  return 0
    else:
        classical,quantumstate = quantumstate.split("?",1)
        classical = classical.split(".")
        lastdamaged=classical.pop() # To be able to continue the list of damaged vents

        classicalreport = [len(chunk) for chunk in classical if chunk]
        if len(classicalreport)>len(damaged):  return 0
        else: 
            while classicalreport:
                d = damaged.pop(0)
                d2 = classicalreport.pop(0)
                if (d2 != d):  return 0
            res += generatecompatiblestates(lastdamaged+"."+quantumstate,damaged[:])
            res += generatecompatiblestates(lastdamaged+"#"+quantumstate,damaged[:])
            return res

@functools.cache #magic tool with repeat
def smartercompatiblestates(quantumstate,damaged):
    res = 0
    Nneededdamaged = sum(damaged)
    maxdam = max((len(l) for l in re.findall("#+",quantumstate)),default=0) > max(damaged,default=0)
    Nqbits = quantumstate.count("?")
    Ndamaged = quantumstate.count("#")
    if (Ndamaged>Nneededdamaged) or (Ndamaged+Nqbits<Nneededdamaged) or maxdam :  return 0 #Early optimisation
    
    if Nqbits==0:
        return int(tuple([len(chunk) for chunk in quantumstate.split(".") if chunk]) == damaged)
    else:
        loc = quantumstate.find("?",Nqbits//2)
    
        qsa,qsb = quantumstate[:loc],quantumstate[loc+1:]

        for k in range(len(damaged)+1): 
            a = smartercompatiblestates(qsa,damaged[:k])
            if a>0: res += a*smartercompatiblestates(qsb,damaged[k:]) #best idea in the code

        qs1 = qsa+"#"+qsb
        res+= smartercompatiblestates(qs1,damaged)
    return res

    

# Part 1

In [83]:
with open("input12.txt") as f:
    logs = [SpringReport.fromrow(line) for line in f.read().splitlines()]


In [72]:
print(f"Number of combinations: {sum([l.totalnumcombinations() for l in logs])}")

Number of combinations: 5028624


In [68]:
%time print(f"Number of compatible combinations: {sum([l.dumbcompatiblestates() for l in logs])}")

Number of compatible combinations: 7732
CPU times: user 4.73 s, sys: 53.4 ms, total: 4.78 s
Wall time: 4.78 s


In [73]:
%time print(f"Number of compatible combinations: {sum([l.smartcompatible()for l in logs])}")

Number of compatible combinations: 7732
CPU times: user 125 ms, sys: 2.23 ms, total: 127 ms
Wall time: 126 ms


In [84]:
%time print(f"Number of compatible combinations: {sum([l.smartercompatible()for l in logs])}")

Number of compatible combinations: 7732
CPU times: user 465 ms, sys: 7.33 ms, total: 472 ms
Wall time: 471 ms


# Part 2

In [88]:
with open("input12.txt") as f:
    logs5 = [SpringReport.fromrow(line,repeat=5) for line in f.read().splitlines()]

In [76]:
ex = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
"""
ex5 = [SpringReport.fromrow(line,repeat=5) for line in ex.splitlines()]
ex1 = [SpringReport.fromrow(line,repeat=1) for line in ex.splitlines()]

In [77]:
print(f"Number of combinations: {sum([l.totalnumcombinations() for l in logs5])}")

Number of combinations: 2313767343144790502270828544


In [78]:
from tqdm.auto import tqdm

In [89]:
%%time 
res =[]

for l in tqdm(logs5):
    res.append(l.smartercompatible())
print(f"Number of compatible combinations: {sum(res)}")

  0%|          | 0/1000 [00:00<?, ?it/s]

Number of compatible combinations: 4500070301581
CPU times: user 23.9 s, sys: 576 ms, total: 24.5 s
Wall time: 24.5 s
